# Prepare data label for CNN model

## Environment setup

In [1]:
# Setup 

DATA_PATH = 'data/'
IMAGES_PATH = 'data/formula_images/'

FORMULAS = DATA_PATH + 'im2latex_formulas.lst'

In [2]:
# Import

from sys import stdout
from PIL import Image
import os
import PIL.ImageOps 
import numpy as np
import cPickle as pickle
import nltk
import scipy.misc
import scipy.sparse

from tokenizer import Tokenizer

## Label with formula tokenization
目前將'command' token直接載入，將'text' token以character方式載入，還未做過濾，有些'command'與'text'不會顯示在圖片上。

### Option 1: No filter
目前將'command' token直接載入，將'text' token以character方式載入，還未做過濾，有些'command'與'text'不會顯示在圖片上。

In [3]:
# build formula list

with open(FORMULAS) as formulas_file:
    formula_list = []
    for line in formulas_file:
        formula_list.append(line)

In [7]:
print formula_list[0]

\int_{-\epsilon}^\infty dl\: {\rm e}^{-l\zeta}	\int_{-\epsilon}^\infty dl' {\rm e}^{-l'\zeta}	ll'{l'-l \over l+l'} \{3\,\delta''(l) - {3 \over 4}t\,\delta(l) \} =0.		\label{eq21}



In [10]:
[i for i in Tokenizer(formula_list[0]).tokenize()]

[(1, 'command', 'int', '\\int'),
 (1, 'text', '_', '_'),
 (1, 'bgroup', '{', '{'),
 (1, 'text', '-', '-'),
 (1, 'command', 'epsilon', '\\epsilon'),
 (1, 'egroup', '}', '}'),
 (1, 'text', '^', '^'),
 (1, 'command', 'infty', '\\infty '),
 (1, 'text', 'dl', 'dl'),
 (1, 'command', ':', '\\:'),
 (1, 'text', ' ', ' '),
 (1, 'bgroup', '{', '{'),
 (1, 'command', 'rm', '\\rm '),
 (1, 'text', 'e', 'e'),
 (1, 'egroup', '}', '}'),
 (1, 'text', '^', '^'),
 (1, 'bgroup', '{', '{'),
 (1, 'text', '-l', '-l'),
 (1, 'command', 'zeta', '\\zeta'),
 (1, 'egroup', '}', '}'),
 (1, 'text', '\t', '\t'),
 (1, 'command', 'int', '\\int'),
 (1, 'text', '_', '_'),
 (1, 'bgroup', '{', '{'),
 (1, 'text', '-', '-'),
 (1, 'command', 'epsilon', '\\epsilon'),
 (1, 'egroup', '}', '}'),
 (1, 'text', '^', '^'),
 (1, 'command', 'infty', '\\infty '),
 (1, 'text', "dl' ", "dl' "),
 (1, 'bgroup', '{', '{'),
 (1, 'command', 'rm', '\\rm '),
 (1, 'text', 'e', 'e'),
 (1, 'egroup', '}', '}'),
 (1, 'text', '^', '^'),
 (1, 'bgroup', '

In [5]:
# tokenize formula list

formula_tokens_list = []

for formula in formula_list:
    tokens_dict = {}
    tokenizer = Tokenizer(formula)
    for token in tokenizer.tokenize():
        # formation of token: (line_number, type, token_part, token_full)        
        # command token process
        if token[1] == 'command':
            if '\\'+token[2] not in tokens_dict:
                tokens_dict['\\'+token[2]] = 1
            else:
                tokens_dict['\\'+token[2]] += 1
        # text token process
        elif token[1] == 'text':
            for c in token[3]:
                if c not in tokens_dict:
                    tokens_dict[c] = 1
                else:
                    tokens_dict[c] += 1
    formula_tokens_list.append(tokens_dict)

In [6]:
print formula_tokens_list[0]

{'\t': 4, '-': 6, '\\epsilon': 2, '\n': 1, '\\zeta': 2, '\\delta': 2, ' ': 9, '\\infty': 2, '\\over': 2, ')': 2, '(': 2, '+': 1, '\\:': 1, '.': 1, '1': 1, '0': 1, '3': 2, '2': 1, '4': 1, '\\,': 2, '\\int': 2, '=': 1, '\\{': 1, '\\label': 1, "'": 7, '\\rm': 2, '_': 2, '^': 4, 'e': 3, 'd': 2, '\\}': 1, 'l': 12, 'q': 1, 't': 1}


In [11]:
# build token dictionary

tokens_count = {}
tokens_dict = {}

# count token frequency
for formula_tokens in formula_tokens_list:
    for token, value in formula_tokens.iteritems():
        if token not in tokens_count:
            tokens_count[token] = value
        else:
            tokens_count[token] += 1

#build token dictionary according to token frequency
index = 2 # 0 for '<eos>' 1 for 'UNK'
tokens_sorted = sorted(tokens_count, key = tokens_count.get, reverse = True)
for token in tokens_sorted:
    tokens_dict[token] = index
    index += 1

In [6]:
# build input file

with open(DATA_PATH + 'cnn_token_dictionary.pkl', 'wb') as f:
    pickle.dump(tokens_dict, f)

NameError: name 'dataPath' is not defined

### Option 2: Filter
將一些看不到的過濾掉。

In [12]:
# build formula list

with open(FORMULAS) as formulas_file:
    formula_list = []
    for line in formulas_file:
        formula_list.append(line)

In [13]:
# tokenize formula list

formula_tokens_list = []

for formula in formula_list:
    tokens_dict = {}
    tokenizer = Tokenizer(formula)
    for token in tokenizer.tokenize():
        # formation of token: (line_number, type, token_part, token_full)        
        # command token process
        if token[1] == 'command':
            if '\\'+token[2] not in tokens_dict:
                tokens_dict['\\'+token[2]] = 1
            else:
                tokens_dict['\\'+token[2]] += 1
        # text token process
        elif token[1] == 'text':
            for c in token[3]:
                if c not in tokens_dict:
                    tokens_dict[c] = 1
                else:
                    tokens_dict[c] += 1
    formula_tokens_list.append(tokens_dict)

In [17]:
# build token dictionary

tokens_count = {}
tokens_dict = {}

# count token frequency
for formula_tokens in formula_tokens_list:
    for token, value in formula_tokens.iteritems():
        if token not in tokens_count:
            tokens_count[token] = value
        else:
            tokens_count[token] += 1

# remove tokens
for key in tokens_count.keys():
    if key in tokens_remove:
        tokens_count.pop(key, None)

#build token dictionary according to token frequency
index = 2 # 0 for '<eos>' 1 for 'UNK'
tokens_sorted = sorted(tokens_count, key = tokens_count.get, reverse = True)
for token in tokens_sorted:
    tokens_dict[token] = index
    index += 1

In [16]:
# tokens to be removed
tokens_remove = ['\n', '_', '^', '\\label', ',', '\\left', '\\right', '\\cal', 'rm', '\\;', '\\ ', '\\\\', '\\begin','\\end',
                 '\\quad', '&', '\\qquad', '\\bf', '\\mbox', '\\mathcal', '\!', '\t', '\\hspace', '\\:', '\\cr',
                 '\\displaystyle', '\\Big', '\\mathrm', '\\stackrel', '\\mid', '\\Bigr', '\\mathbf', '\\biggr', '\\biggl',
                 '\\hbox', '\\textstyle', '\\big', '\\pmatrix', '\\bigr', '\\bigl', '\\matrix', '\r', '\\bigg', '\\hskip', '\\>',
                 '\\boldmath', '\\textrm', '\\atop', '\\Bigg', '\\Biggr', '\\Biggl', '\\scriptsize', '\\it',
                 '\\scriptscriptstyle', '\\sp', '\\scriptstyle', '\\tiny', '\\phantom', '\\hline', '\\choose', '\\', '\\small',
                 '\\sf', '\\cases', '\\vspace', '\\kern', '\\put', '\\hspace*', '\\makebox', '\\raisebox', '\\mit', '\\protect',
                 '\\line', '\\large', '\\unitlength', '\\vphantom', '@']
special = ['\\label', '\\begin', '\\end', '\\hspace', '\\hskip', '\\phantom', '\\hline', '\\choose', '\\vspace', '\\kern',
           '\\put', '\\hspace*', '\\raisebox', '\\line', '\\unitlength', '\\vphantom', '@']
change_style = ['\\cal', '\\bf', '\\rm', '\\mathcal', '\\displaystyle', '\\Big', '\\stackrel', '\\Bigr', '\\mathbf', '\\biggr',
                '\\biggl', '\\textstyle', '\\big', '\\bigr', '\\bigl', '\\bigg', '\\boldmath', '\\textrm', '\\atop',
                '\\Bigg', '\\Biggr', '\\Biggl', '\\scriptsize', '\\it', '\\scriptscriptstyle', '\\sp', '\\scriptstyle', '\\tiny',
                '\\small', '\\sf', '\\mit', '\\large']

In [20]:
print tokens_dict

{'\\perp': 172, '\\leq': 163, '\\setminus': 317, '\\supset': 245, ' ': 3, '\xab': 484, '0': 14, '\\Re': 230, '\\smallint': 455, '\\nearrow': 362, '\\surd': 363, '\\vert': 160, '\\rlap': 332, '7': 111, 'P': 77, '\\textbf': 284, '\\of': 485, '\\overbrace': 299, '\\verb': 456, '\\Sigma': 138, '\\rho': 87, '\\tilde': 82, '`': 269, '\\mathopen': 392, '\\limsup': 457, '\\mathbin': 393, 'p': 27, '?': 318, '\\asymp': 394, '\\Psi': 128, '\\newcommand': 486, '\\Omega': 112, '\xa0': 373, '\\nabla': 133, '+': 9, '\\Longrightarrow': 224, '\\mathit': 298, '\\coprod': 487, '\\emptyset': 281, ';': 106, '\\colon': 295, '\\bigcup': 289, 'K': 93, '\\ensuremath': 412, '\\leftarrow': 250, '\\prec': 424, '\\rm': 68, '\\ointop': 458, '\\bigoplus': 210, '\\aleph': 300, '\\nolimits': 302, '\\cdots': 132, '\\cdotp': 253, 'k': 36, '\\nonumber': 178, '\\swarrow': 395, '\\cong': 206, '\\cos': 142, '\\AA': 396, '\\equiv': 104, '\\widetilde': 152, '\\ln': 129, '\\ll': 203, '\\huge': 459, '\\le': 188, '\\star': 168, 

In [13]:
# build input file

with open(DATA_PATH + 'cnn_token_filtered_dictionary.pkl', 'wb') as f:
    pickle.dump(tokens_dict, f)

In [21]:
print len(tokens_sorted)
tokens_sorted

522


['=',
 ' ',
 '(',
 ')',
 '2',
 '1',
 '-',
 '+',
 '.',
 'i',
 'e',
 '\\frac',
 '0',
 'a',
 'd',
 'r',
 'n',
 't',
 'c',
 'm',
 'x',
 '3',
 '4',
 '\\,',
 's',
 'p',
 'g',
 'l',
 'q',
 '\\mu',
 '\\pi',
 'o',
 'A',
 'f',
 'k',
 '\\partial',
 '\\int',
 'y',
 'b',
 '\\alpha',
 '\\over',
 'S',
 'u',
 'j',
 'T',
 '\\delta',
 '\\sum',
 'D',
 'R',
 'h',
 ':',
 '\\phi',
 'L',
 'N',
 'M',
 '\\sqrt',
 'F',
 '\\nu',
 'B',
 '|',
 '\\lambda',
 '/',
 'z',
 '5',
 '\\beta',
 'H',
 '\\rm',
 '\\bar',
 "'",
 'G',
 'C',
 '6',
 'V',
 'v',
 'E',
 'P',
 '\\gamma',
 '\\sigma',
 '\\{',
 '\\theta',
 '\\tilde',
 '\\epsilon',
 '8',
 'I',
 '\\}',
 '\\rho',
 '\\psi',
 '\\infty',
 '\\omega',
 '\\hat',
 '\\tau',
 'K',
 'J',
 '\\eta',
 'Q',
 'X',
 'w',
 'W',
 '\\Gamma',
 'Z',
 '\\rangle',
 '\\Phi',
 '\\equiv',
 '\\Delta',
 ';',
 '\\Lambda',
 '\\xi',
 'U',
 '*',
 '7',
 '\\Omega',
 '\\dot',
 '\\varphi',
 '\\cdot',
 '\\prime',
 '\\langle',
 '\\chi',
 '\\kappa',
 '9',
 '\\pm',
 '\\exp',
 '\\vec',
 '>',
 'O',
 '\\rightarrow',

## Images data process

### Option 1: Raw Image
只將圖片載入成sparse matrix，生成im2latex_feature.train.pkl、im2latex_feature.validate.pkl與im2latex_feature.test.pkl。

In [ ]:
# write image pkl
# image shape: (1654,2339)

files = ['validate', 'test', 'train']

for fname in files:
    print fname 
    f = open(DATA_PATH + 'im2latex_' + fname + '.lst')
    counter = 1
    
    imageList = [i.split()[1] + '.png' for i in f]
    numImage = len(imageList)

    image = np.rollaxis(scipy.misc.imread(IMAGES_PATH+imageList[0], flatten = True),1)
    inverseImage = 255 - image
    reshapeImage = np.reshape(inverseImage,(1,-1))
    sparseImage = scipy.sparse.csr_matrix(reshapeImage)
    result = sparseImage

    for i in range(1,numImage):
        image = np.rollaxis(scipy.misc.imread(IMAGES_PATH+imageList[i], flatten = True),1)
        inverseImage = 255 - image
        reshapeImage = np.reshape(inverseImage,(1,-1))
        sparseImage = scipy.sparse.csr_matrix(reshapeImage)
        
        counter += 1
        stdout.write("\r%d" % counter)
        stdout.flush()
        result = scipy.sparse.vstack((result, sparseImage))
        
    print result.shape
    
    resultSave32 = result.astype('float32')
    fileName = open(dataPath + 'im2latex_feature.' + fname + '.pkl','wb')
    pickle.dump(resultSave32, fileName ,-1)
    fileName.close()

### Option 2: Image cutting
將圖片載入成numpy ndarray，並且切除周圍空白區域，生成im2latex_image_cut.train.pkl、im2latex_image_cut.validate.pkl與im2latex_image_cut.test.pkl。

In [7]:
# retrive the max region of formulas in images

top_most = 500
bottom_most = 500
left_most = 827
right_most = 827

counter = 0

images_file = os.listdir(IMAGES_PATH)

for image_file in images_file:
    image = scipy.misc.imread(IMAGES_PATH + image_file, flatten = True)
    inverse = 255-image
    nonzero = np.argwhere(inverse)
    if nonzero.size:
        (top, left), (bottom, right) = nonzero.min(0), nonzero.max(0) + 1
    
    if top < top_most: top_most = top
    if bottom > bottom_most: bottom_most = bottom
    if left < left_most: left_most = left
    if right > right_most: right_most = right
    
    counter += 1
    stdout.write("\r%d" % counter)
    stdout.flush()
print (top_most, left_most), (bottom_most, right_most)

103536(392, 109) (1880, 1654)


In [8]:
# show the region
print (top_most, left_most), (bottom_most, right_most)

(392, 109) (1880, 1654)


In [ ]:
# write image pkl

WIDTH = 1132 - 558
HEIGHT = 577 - 419


files = ['test', 'train']

for fname in files:
    print fname 
    f = open(DATA_PATH + 'im2latex_' + fname + '.lst')
    counter = 0
    
    image_list = [i.split()[1] + '.png' for i in f]
    image_number = len(image_list)
    
    result = np.empty((image_number, HEIGHT, WIDTH))

    for i in range(0,image_number):
        image = scipy.misc.imread(IMAGES_PATH+image_list[i], flatten = True)
        image = image[419:577,558:1132]
        image_inverse = 255 - image
        
        counter += 1
        stdout.write("\r%d" % counter)
        stdout.flush()
        result[i,:,:] = image_inverse
        
    print result.shape
    
    result_save32 = result.astype('float32')
    np.save(DATA_PATH + 'im2latex_image_cut.' + fname + '.npy', result_save32)
    

test
10355(10355L, 158L, 574L)


In [11]:
import matplotlib.pyplot as plt
image = scipy.misc.imread(IMAGES_PATH+'1a00a76d4e.png', flatten = True)
image = image[419:577,558:1132]
image_inverse = 255 - image

image2 = scipy.misc.imresize(image_inverse,33)
print image2.shape


plt.imshow(image2)
plt.show()

(52L, 189L)


## Align image with label
將label與image對齊，生成cnn_align.train.pkl、cnn_align.validate.pkl、cnn_align.test.pkl。
要用時，先載入label list，再載入image list (list of sparse matrix)。

In [15]:
# write align pkl

files = ['validate', 'test', 'train']

for name in files:
    counter = 0
    images = pickle.load(open(DATA_PATH + 'im2latex_feature.' + name +'.pkl','rb'))
    examples = open(DATA_PATH + 'im2latex_' + name + '.lst')
    labels = []
    for example in examples:
        label = formula_tokens_list[int(example.split()[0])]
        labels.append(label)
        counter += 1
    saveFile = open(DATA_PATH + 'cnn_align.' + name + '.pkl', 'wb')
    pickle.dump(labels, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    pickle.dump(images, saveFile, protocol=pickle.HIGHEST_PROTOCOL)
    saveFile.close()

# Some test

In [ ]:
tokens_dict['=']

In [ ]:
tokens_count

In [11]:
tokens_sorted

['\n',
 '=',
 '_',
 ' ',
 '^',
 '(',
 ')',
 '2',
 '1',
 '-',
 '\\label',
 ',',
 '+',
 '.',
 'i',
 'e',
 '\\frac',
 '0',
 'a',
 'd',
 'r',
 'n',
 '\\left',
 '\\right',
 't',
 'c',
 'm',
 'x',
 '3',
 '4',
 '\\,',
 's',
 'p',
 'g',
 'l',
 'q',
 '\\mu',
 '\\pi',
 'o',
 'A',
 'f',
 'k',
 '\\partial',
 '\\int',
 'y',
 'b',
 '\\alpha',
 '\\over',
 'S',
 'u',
 '\\cal',
 'j',
 'T',
 '\\;',
 '\\delta',
 '\\sum',
 'D',
 'R',
 '\\ ',
 'h',
 ':',
 '\\phi',
 'L',
 'N',
 'M',
 '\\sqrt',
 'F',
 '\\nu',
 'B',
 '|',
 '\\lambda',
 '/',
 'z',
 '5',
 '\\beta',
 'H',
 '\\rm',
 '\\bar',
 "'",
 'G',
 'C',
 '6',
 'V',
 'v',
 'E',
 'P',
 '\\gamma',
 '\\sigma',
 '\\{',
 '\\theta',
 '\\\\',
 '\\tilde',
 '\\epsilon',
 '8',
 'I',
 '\\begin',
 '\\end',
 '\\}',
 '\\rho',
 '\\psi',
 '\\quad',
 '\\infty',
 '\\omega',
 '\\hat',
 '\\tau',
 'K',
 'J',
 '\\eta',
 'Q',
 '&',
 'X',
 'w',
 'W',
 '\\Gamma',
 'Z',
 '\\rangle',
 '\\Phi',
 '\\equiv',
 '\\Delta',
 ';',
 '\\Lambda',
 '\\qquad',
 '\\xi',
 'U',
 '*',
 '7',
 '\\bf',
 

In [ ]:
# tokens filter
tokens_remove = ['\n', '_', '^', '\\label', ',', '\\left', '\\right', '\\cal', 'rm', '\\;', '\\ ', '\\\\', '\\begin','\\end',
                 '\\quad', '&', '\\qquad', '\\bf', '\\mbox', '\\mathcal', '\!', '\t', '\\hspace', '\\:', '\\cr',
                 '\\displaystyle', '\\Big', '\\mathrm', '\\stackrel', '\\mid', '\\Bigr', '\\mathbf', '\\biggr', '\\biggl',
                 '\\hbox', '\\textstyle', '\\big', '\\pmatrix', '\\bigr', '\\bigl', \\matrix, '\r', '\\bigg', '\\hskip', '\\>',
                 '\\boldmath', '\\textrm', '\\atop', '\\Bigg', '\\Biggr', '\\Biggl', '\\scriptsize', '\\it',
                 '\\scriptscriptstyle', '\\sp', '\\scriptstyle', '\\tiny', '\\phantom', '\\hline', '\\choose', '\\', '\\small',
                 '\\sf', '\\cases', '\\vspace', '\\kern', '\\put', '\\hspace*', '\\makebox', '\\raisebox', '\\mit', '\\protect',
                 '\\line', '\\large', '\\unitlength', '\\vphantom', '@']
special = ['\\label', '\\begin', '\\end', '\\hspace', '\\hskip', '\\phantom', '\\hline', '\\choose', '\\vspace', '\\kern',
           '\\put', '\\hspace*', '\\raisebox', '\\line', '\\unitlength', '\\vphantom', '@']
change_style = ['\\cal', '\\bf', '\\rm', '\\mathcal', '\\displaystyle', '\\Big', '\\stackrel', '\\Bigr', '\\mathbf', '\\biggr',
                '\\biggl', '\\textstyle', '\\big', '\\bigr', '\\bigl', '\\bigg', '\\boldmath', '\\textrm', '\\atop',
                '\\Bigg', '\\Biggr', '\\Biggl', '\\scriptsize', '\\it', '\\scriptscriptstyle', '\\sp', '\\scriptstyle', '\\tiny',
                '\\small', '\\sf', '\\mit', '\\large']

In [ ]:
formula_tokens_list[1]